In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal

In [12]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END

# Define the state
class QuadState(TypedDict):
    a: float
    b: float
    c: float
    discriminant: float
    roots: list[float]
    message: str

In [13]:
def show_equation(state: QuadState) -> QuadState:
    state['message'] = f"The equation is {state['a']}x^2 + {state['b']}x + {state['c']} = 0"
    return state

# --- Original code returned a tuple instead of calculating discriminant ---
def calculate_descriminant(state: QuadState) -> QuadState:
    # Original error:
    # discriminant  = state['a'], state['b'], state['c']  # ❌ tuple, not formula
    a = state['a']
    b = state['b']
    c = state['c']
    discriminant = b**2 - 4*a*c
    state['discriminant'] = discriminant
    return state

# --- Original code returned string instead of state ---
def real_root(state: QuadState) -> QuadState:
    d = state['discriminant']
    a = state['a']
    b = state['b']
    root1 = (-b + d**0.5) / (2*a)
    root2 = (-b - d**0.5) / (2*a)
    state['roots'] = [root1, root2]
    state['message'] = f"The roots are real and different. Root1 = {root1}, Root2 = {root2}"
    return state

def repeated_root(state: QuadState) -> QuadState:
    a = state['a']
    b = state['b']
    root = -b / (2*a)
    state['roots'] = [root]
    state['message'] = f"The roots are real and same. Root = {root}"
    return state

def no_real_root(state: QuadState) -> QuadState:
    state['message'] = "The roots are complex and different."
    return state

# --- Original function was fine, just make sure it uses 'discriminant' correctly ---
def check_condition(state: QuadState) -> Literal["real_roots", "repeated_roots", "no_real_roots"]:
    d = state['discriminant']
    if d > 0:
        return "real_roots"
    elif d == 0:
        return "repeated_roots"
    else:
        return "no_real_roots"


In [14]:
graph = StateGraph(QuadState)

graph.add_node('show_equation', show_equation)
graph.add_node('calculate_descriminant', calculate_descriminant)
graph.add_node('real_roots', real_root)
graph.add_node('repeated_roots', repeated_root)
graph.add_node('no_real_roots', no_real_root)

graph.add_edge(START, 'show_equation')
graph.add_edge('show_equation', 'calculate_descriminant')

# --- Original error: typo in node name ---
# graph.add_conditional_edges('calculate_discriminant', check_condition)  # ❌ wrong node name
graph.add_conditional_edges('calculate_descriminant', check_condition)    # ✅ correct node name

graph.add_edge('real_roots', END)
graph.add_edge('repeated_roots', END)
graph.add_edge('no_real_roots', END)

workflow = graph.compile()

# ---------------- Test ----------------
initial_state = QuadState(a=1, b=-3, c=2, discriminant=0.0, roots=[], message="")
final_state = workflow.invoke(initial_state)
print(final_state['message'])
print(final_state.get('roots'))


The roots are real and different. Root1 = 2.0, Root2 = 1.0
[2.0, 1.0]
